Учебно-технологическая практика Некиров М.В. К3-22Б

In [7]:
%matplotlib inline
from datetime import datetime
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
from bisect import bisect
from matplotlib.colors import Normalize
from pathlib import Path
import os

In [31]:
def load_dataset(path: Path) -> pd.DataFrame:
    df = pd.read_json(path)
    df.date = pd.to_datetime(df.date, format="%Y-%m-%d %H:%M:%S")
    df.set_index('date', inplace=True)
    return df

ROOT = Path(os.getcwd()).resolve()

dataframes = {}
for file in os.listdir('datasets'):
    path = ROOT / 'datasets' / file
    print(file, path)
    dataframes[path.stem] = load_dataset(path)
print('OK')

Hydra-L.json C:\Users\idiya\practice\datasets\Hydra-L.json
Hydra-L1.json C:\Users\idiya\practice\datasets\Hydra-L1.json
reference.json C:\Users\idiya\practice\datasets\reference.json
Опорный барометр.json C:\Users\idiya\practice\datasets\Опорный барометр.json
Паскаль.json C:\Users\idiya\practice\datasets\Паскаль.json
РОСА К-2.json C:\Users\idiya\practice\datasets\РОСА К-2.json
Сервер dbrobo.json C:\Users\idiya\practice\datasets\Сервер dbrobo.json
Сервер dokuwiki.json C:\Users\idiya\practice\datasets\Сервер dokuwiki.json
Сервер K3edu.json C:\Users\idiya\practice\datasets\Сервер K3edu.json
Сервер webrobo.json C:\Users\idiya\practice\datasets\Сервер webrobo.json
Сервер СЕВ.json C:\Users\idiya\practice\datasets\Сервер СЕВ.json
Тест воздуха.json C:\Users\idiya\practice\datasets\Тест воздуха.json
Тест СБ.json C:\Users\idiya\practice\datasets\Тест СБ.json
Тест Студии.json C:\Users\idiya\practice\datasets\Тест Студии.json
OK


In [3]:
def zscore_cleaning(data) -> pd.DataFrame:
    z = np.abs(stats.zscore(data))
    data = data[(z < 3).all(axis=1)]
    return data


def quantile_cleaning(data) -> pd.DataFrame:
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = data.apply(stats.iqr)
    data = data[~((data < (Q1 - 1.5 * IQR)) | (data > (Q3 + 1.5 * IQR))).any(axis=1)]
    return data


def draw_mean(data: pd.DataFrame, window: int | str = '1h') -> None:
    data = data.dropna()
    res = data.resample(window).mean()
    plt.plot(data)
    plt.plot(res, color='red')
    plt.xticks(rotation=70)
    plt.plot()
    plt.show()


def min_max_candle(data: pd.DataFrame) -> None:
    gb = data.dropna().groupby(pd.Grouper(freq='d'))
    Y = [np.array(gb.get_group(x)) for x in gb.groups]
    X = [x for x in gb.groups]
    plt.boxplot(Y, labels=X, showfliers=False)
    _ = plt.xticks(rotation=70)


def effective_temp(t: np.array, h: np.array) -> np.array:
    return t - 0.4 * (t - 10) * (1 - h / 100)


HEAT_PERCEPTION = {-30: 'Крайне холодно', -24: 'Крайне холодно', -12: 'Очень холодно', 0: 'Холодно',
                   12: 'Умеренно тепло', 18: 'Тепло', 24: 'Жарко', 30: 'Очень жарко'}

_HP_VALUES = list(HEAT_PERCEPTION)
_HP_LABELS = list(HEAT_PERCEPTION.values())


def convert_labels(x: int) -> int:
    return bisect(_HP_VALUES, x)


def plot_effective(t: np.ndarray, h: np.ndarray) -> None:
    fig, (ax1, ax2) = plt.subplots(2, 1)
    fig.autofmt_xdate(rotation=70)

    eff = effective_temp(t, h)

    ax1.plot(t, label='temperature')
    ax1.plot(eff, label='effective temperature')
    ax1.set_ylabel(r'Temperature ($^\circ$C)')

    x = [convert_labels(x) for x in eff]
    colormap = plt.get_cmap("turbo")
    rescale = Normalize(np.min(_HP_VALUES), np.max(_HP_VALUES))

    ax2.bar(eff.index, x, width=25 / len(x), label='heat perception', color=colormap(rescale(eff)))
    ax2.yaxis.tick_right()
    ax2.set_ylabel('Heat perception')
    ax2.set_yticks(ticks=np.arange(len(_HP_VALUES)), labels=_HP_LABELS)
    fig.legend()

 Интерактивная клетка: выбор приборов, данных, типа графика, временной промежуток наблюдения, усреднение за промежуток

In [30]:
meanButtons = widgets.RadioButtons(options=['1h', '3h', '1d'], description='Averaging window:')
action = widgets.RadioButtons(options=['Real Data', 'Average', 'Min/Max'], description='Action:')
dfDropdown = widgets.Dropdown(options=list(dataframes), description='source')
columnDropdown = widgets.Dropdown(options=dataframes[list(dataframes)[0]], description='data')

start_date, end_date = datetime(2022, 7, 2), datetime(2022, 7, 30)
dates = pd.date_range(start_date, end_date, freq='D')
options = [(date.strftime(' %d %b %Y '), date) for date in dates]
selection_range_slider = widgets.SelectionRangeSlider(
    options=options,
    value=(start_date, end_date),
    index=(0, len(options) - 1),
    description='Dates',
    orientation='horizontal',
    layout={'width': '500px'}
)


def update_columns(change):
    owner = change['owner'].value
    l = dataframes[owner].columns.tolist()
    columnDropdown.options = l
    columnDropdown.value = l[0]


dfDropdown.observe(update_columns)


def printer(action, meanButtons, df, clmn, selection_range_slider):
    s, e = selection_range_slider
    df = dataframes[df][clmn]
    mask = (df.index > s) & (df.index <= e)
    match action:
        case 'Real Data':
            df.loc[mask].plot()
        case 'Average':
            draw_mean(df.loc[mask], meanButtons)
        case 'Min/Max':
            min_max_candle(df.loc[mask])


widgets.interact(printer, meanButtons=meanButtons, selection_range_slider=selection_range_slider, action=action,
                 df=dfDropdown, clmn=columnDropdown);

interactive(children=(RadioButtons(description='Action:', options=('Real Data', 'Average', 'Min/Max'), value='…

График эффективной температуры и теплоощущения

In [24]:
effective_df = {'Hydra-L': dataframes['Hydra-L'], 'Hydra-L1': dataframes['Hydra-L1']}

In [26]:
effectiveTempDropdown = widgets.Dropdown(options=list(effective_df))


def effective_printer(meanButtons, frame):
    df = effective_df[frame]
    df = df.resample(meanButtons).mean()
    plot_effective(df.BME280_temp, df.BME280_humidity)


widgets.interact(effective_printer, meanButtons=meanButtons, frame=effectiveTempDropdown);

interactive(children=(RadioButtons(description='Averaging window:', options=('1h', '3h', '1d'), value='1h'), D…